# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fbb1c8e1d00>
├── 'a' --> tensor([[-0.5244, -0.7549, -1.1202],
│                   [-1.8828, -0.0444,  0.1735]])
└── 'x' --> <FastTreeValue 0x7fbaf45bb3a0>
    └── 'c' --> tensor([[-1.8726, -0.2404, -1.6999, -1.8528],
                        [ 0.0086, -0.9680,  1.1147,  1.1320],
                        [ 0.5571, -0.9068,  1.7233, -0.6001]])

In [4]:
t.a

tensor([[-0.5244, -0.7549, -1.1202],
        [-1.8828, -0.0444,  0.1735]])

In [5]:
%timeit t.a

62.6 ns ± 0.0853 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fbb1c8e1d00>
├── 'a' --> tensor([[-0.5495,  0.4204, -0.0300],
│                   [ 1.3260,  1.4084,  0.4304]])
└── 'x' --> <FastTreeValue 0x7fbaf45bb3a0>
    └── 'c' --> tensor([[-1.8726, -0.2404, -1.6999, -1.8528],
                        [ 0.0086, -0.9680,  1.1147,  1.1320],
                        [ 0.5571, -0.9068,  1.7233, -0.6001]])

In [7]:
%timeit t.a = new_value

67.1 ns ± 0.0497 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.5244, -0.7549, -1.1202],
               [-1.8828, -0.0444,  0.1735]]),
    x: Batch(
           c: tensor([[-1.8726, -0.2404, -1.6999, -1.8528],
                      [ 0.0086, -0.9680,  1.1147,  1.1320],
                      [ 0.5571, -0.9068,  1.7233, -0.6001]]),
       ),
)

In [10]:
b.a

tensor([[-0.5244, -0.7549, -1.1202],
        [-1.8828, -0.0444,  0.1735]])

In [11]:
%timeit b.a

60.3 ns ± 0.0276 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.3042,  0.8511,  0.7584],
               [ 0.6697,  0.4747,  0.5943]]),
    x: Batch(
           c: tensor([[-1.8726, -0.2404, -1.6999, -1.8528],
                      [ 0.0086, -0.9680,  1.1147,  1.1320],
                      [ 0.5571, -0.9068,  1.7233, -0.6001]]),
       ),
)

In [13]:
%timeit b.a = new_value

492 ns ± 6.49 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

814 ns ± 0.0759 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 50.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 339 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

146 µs ± 281 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fbb1c8d8c40>
├── 'a' --> tensor([[[-0.5244, -0.7549, -1.1202],
│                    [-1.8828, -0.0444,  0.1735]],
│           
│                   [[-0.5244, -0.7549, -1.1202],
│                    [-1.8828, -0.0444,  0.1735]],
│           
│                   [[-0.5244, -0.7549, -1.1202],
│                    [-1.8828, -0.0444,  0.1735]],
│           
│                   [[-0.5244, -0.7549, -1.1202],
│                    [-1.8828, -0.0444,  0.1735]],
│           
│                   [[-0.5244, -0.7549, -1.1202],
│                    [-1.8828, -0.0444,  0.1735]],
│           
│                   [[-0.5244, -0.7549, -1.1202],
│                    [-1.8828, -0.0444,  0.1735]],
│           
│                   [[-0.5244, -0.7549, -1.1202],
│                    [-1.8828, -0.0444,  0.1735]],
│           
│                   [[-0.5244, -0.7549, -1.1202],
│                    [-1.8828, -0.0444,  0.1735]]])
└── 'x' --> <FastTreeValue 0x7fbb1c8d8ca0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 74 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fba66195ca0>
├── 'a' --> tensor([[-0.5244, -0.7549, -1.1202],
│                   [-1.8828, -0.0444,  0.1735],
│                   [-0.5244, -0.7549, -1.1202],
│                   [-1.8828, -0.0444,  0.1735],
│                   [-0.5244, -0.7549, -1.1202],
│                   [-1.8828, -0.0444,  0.1735],
│                   [-0.5244, -0.7549, -1.1202],
│                   [-1.8828, -0.0444,  0.1735],
│                   [-0.5244, -0.7549, -1.1202],
│                   [-1.8828, -0.0444,  0.1735],
│                   [-0.5244, -0.7549, -1.1202],
│                   [-1.8828, -0.0444,  0.1735],
│                   [-0.5244, -0.7549, -1.1202],
│                   [-1.8828, -0.0444,  0.1735],
│                   [-0.5244, -0.7549, -1.1202],
│                   [-1.8828, -0.0444,  0.1735]])
└── 'x' --> <FastTreeValue 0x7fba65bca820>
    └── 'c' --> tensor([[-1.8726, -0.2404, -1.6999, -1.8528],
                        [ 0.0086, -0.9680,  1.1147,  1.1320],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 28.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.1 µs ± 49.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.5244, -0.7549, -1.1202],
                [-1.8828, -0.0444,  0.1735]],
       
               [[-0.5244, -0.7549, -1.1202],
                [-1.8828, -0.0444,  0.1735]],
       
               [[-0.5244, -0.7549, -1.1202],
                [-1.8828, -0.0444,  0.1735]],
       
               [[-0.5244, -0.7549, -1.1202],
                [-1.8828, -0.0444,  0.1735]],
       
               [[-0.5244, -0.7549, -1.1202],
                [-1.8828, -0.0444,  0.1735]],
       
               [[-0.5244, -0.7549, -1.1202],
                [-1.8828, -0.0444,  0.1735]],
       
               [[-0.5244, -0.7549, -1.1202],
                [-1.8828, -0.0444,  0.1735]],
       
               [[-0.5244, -0.7549, -1.1202],
                [-1.8828, -0.0444,  0.1735]]]),
    x: Batch(
           c: tensor([[[-1.8726, -0.2404, -1.6999, -1.8528],
                       [ 0.0086, -0.9680,  1.1147,  1.1320],
                       [ 0.5571, -0.9068,  1.7233, -0.6001]],
         

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 609 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.5244, -0.7549, -1.1202],
               [-1.8828, -0.0444,  0.1735],
               [-0.5244, -0.7549, -1.1202],
               [-1.8828, -0.0444,  0.1735],
               [-0.5244, -0.7549, -1.1202],
               [-1.8828, -0.0444,  0.1735],
               [-0.5244, -0.7549, -1.1202],
               [-1.8828, -0.0444,  0.1735],
               [-0.5244, -0.7549, -1.1202],
               [-1.8828, -0.0444,  0.1735],
               [-0.5244, -0.7549, -1.1202],
               [-1.8828, -0.0444,  0.1735],
               [-0.5244, -0.7549, -1.1202],
               [-1.8828, -0.0444,  0.1735],
               [-0.5244, -0.7549, -1.1202],
               [-1.8828, -0.0444,  0.1735]]),
    x: Batch(
           c: tensor([[-1.8726, -0.2404, -1.6999, -1.8528],
                      [ 0.0086, -0.9680,  1.1147,  1.1320],
                      [ 0.5571, -0.9068,  1.7233, -0.6001],
                      [-1.8726, -0.2404, -1.6999, -1.8528],
                      [ 0.0086, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 326 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

326 µs ± 1.36 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
